# Grocery Shopping List Aggregation Test

This notebook checks wether the aggregation that is done on the grocery shopping list work in this particular way:
If the metric and the ingredient name is the same, all items are collected into one row while summing up the respective quantities.

In [1]:
import pandas as pd
from typing import Tuple, List

First lets fetch all the function we need. Make sure to enter a high amount of meals, the meal planner should prepare for you to test the grocery shopping list aggregation. I choose 15.

In [2]:
from inputs.fetch_user_pecific_needs import fetch_user_specific_needs
from inputs.fetch_meals import fetch_available_meals
from processing.meal_planner import get_meal_plan_data_list

For how many days do you want to plan your meals?

okay! let me prepare a meal plan for you...


Lets costumize the output function - meal_plan_presenter - for our notebook here.

In [3]:
TupleType = Tuple[int, str, str]

def present_meal_plan(meal_plan_data_list):
    print("grocery shopping list:")
    ingridients_tuple_list : List(TupleType) = [tup for sublist in meal_plan_data_list[3] for tup in sublist]
    grocery_shopping_list = (
        pd.DataFrame(ingridients_tuple_list, columns=['amount', 'metric', 'ingredient'])
        .dropna(subset=['amount', 'metric', 'ingredient'])
        .astype({'amount': 'int32', 'metric': 'string', 'ingredient': 'string'})
        .groupby(['metric', 'ingredient'], as_index=False).agg({'amount': 'sum'})
    )
    return(grocery_shopping_list)

Lets ececute a part of the code bases code to get a meal plan sample to wotk on.

In [4]:
available_meals = fetch_available_meals()
user_spefifications = fetch_user_specific_needs()
meal_plan_data_list = get_meal_plan_data_list(available_meals, user_spefifications)

In [5]:
pd.set_option('display.max_rows', None)
present_meal_plan(meal_plan_data_list).sort_values(by='ingredient')

grocery shopping list:


,metric,ingredient,amount
55,ml,Caesar dressing,90
21,g,arborio rice,300
56,ml,balsamic vinegar,30
0,,bell pepper,10
1,,bell peppers,4
2,,boiled eggs,6
66,slices,bread,4
22,g,broccoli,600
23,g,bulgur wheat,300
24,g,butter,100


We can observe that the aggregation worked as we intented. But one problem could be spotted. If ingridents are denoted as plural and sigular noun, they wont get aggregated. So lets build a processing that also covers these cases. An exaple would be

| Amount | Ingredient | Count |
|--------|------------|-------|
| 9      | onion      | 10    |
| 10     | onions     | 6     |

In [6]:
def present_meal_plan_adjusted(meal_plan_data_list):
    print("Grocery shopping list:")
    ingredients_tuple_list: List[TupleType] = [tup for sublist in meal_plan_data_list[3] for tup in sublist]
    grocery_shopping_list = (
        pd.DataFrame(ingredients_tuple_list, columns=['amount', 'metric', 'ingredient'])
        .dropna(subset=['amount', 'metric', 'ingredient'])
        .astype({'amount': 'int32', 'metric': 'string', 'ingredient': 'string'})
    )

    # Normalize ingredient names by removing trailing 's'
    grocery_shopping_list['normalized'] = grocery_shopping_list['ingredient'].str.rstrip('s')

    # aggregate the ingredients
    aggregated_grocery_shopping_list = (
        grocery_shopping_list
        .groupby(['metric', 'normalized'], as_index=False).agg({'amount': 'sum'})
    )

    # Strip awas the 's' when amount is 1
    aggregated_grocery_shopping_list['ingredient'] = aggregated_grocery_shopping_list.apply(
        lambda row: row['normalized'] if row['amount'] == 1 else row['normalized'] + 's', axis=1
    )
    aggregated_grocery_shopping_list['ingredient'] = aggregated_grocery_shopping_list.apply(
        lambda row: row['normalized'] if row['metric'] == 'g' else row['normalized'] + 's', axis=1
    )
    aggregated_grocery_shopping_list['ingredient'] = aggregated_grocery_shopping_list.apply(
        lambda row: row['normalized'] if row['metric'] == 'ml' else row['normalized'] + 's', axis=1
    )
    
    aggregated_grocery_shopping_list = aggregated_grocery_shopping_list.drop(columns=['normalized'])

    return aggregated_grocery_shopping_list

In [7]:
present_meal_plan_adjusted(meal_plan_data_list).sort_values(by='ingredient')

Grocery shopping list:


,metric,amount,ingredient
52,ml,90,Caesar dressing
18,g,300,arborio rices
53,ml,30,balsamic vinegar
0,,14,bell peppers
1,,6,boiled eggs
63,slices,4,breads
19,g,600,broccolis
20,g,300,bulgur wheats
21,g,100,butters
2,,3,carrots


# Conclusion
The aggregation works with a few tweeks here and there. The same ingredients are now aggregated into one row and also their ingredient name is adjusted based on the amount and metric of the respective ingredient creating a day to day life grocery shopping list.